In [2]:
import script_04

In [ ]:
from pathlib import Path

In [7]:
script_04.run_rf()

Cached


{('b261',
  'b277'): RFECV(cv=5,
       estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='entropy',
                                        max_depth=None, max_features=0.5,
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=5,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=500, n_jobs=10,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False),
       min_features_to_select=1, n_jobs=-1, scoring=None, step=1, verbose=0),
 ('b261',
  'b278'): RFECV(cv=5,
       estimator=RandomForestClassifier(bootstrap=True

In [3]:
script_04.run_svm_linear()

Cached


{('b261',
  'b277'): RFECV(cv=5,
       estimator=SVC(C=100, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='linear', max_iter=-1,
                     probability=True, random_state=None, shrinking=True,
                     tol=0.001, verbose=False),
       min_features_to_select=1, n_jobs=-1, scoring=None, step=1, verbose=0),
 ('b261',
  'b278'): RFECV(cv=5,
       estimator=SVC(C=100, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='linear', max_iter=-1,
                     probability=True, random_state=None, shrinking=True,
                     tol=0.001, verbose=False),
       min_features_to_select=1, n_jobs=-1, scoring=None, step=1, verbose=0),
 ('b261',
  'b360'): RFECV(cv=5,
       estimator=SVC(C=100, break_ties=False, cache_size=200,